To get started, answer the following questions using just the fullsample.csv jobs dataset:

Calculate some descriptive statistics for how many jobs per hour are being completed. What does the completions per hour look like over the time span of the dataset? Are there weekly trends, and has it been increasing over the last year?

Does the job state affect completions per hour? i.e. if I only look at jobs with exit code 0:0 in the "COMPLETED" state, is that a similar number of completions per hour as with all jobs, failed or cancelled? This will indicate if the load on the scheduler is by user design or is a result of users not sufficiently testing their jobs before submitting very large arrays. We also expect that most job completions will be in the "production" partition, but is this actually true?

In [103]:
import pandas as pd
import datetime

In [11]:
fullsample = pd.read_csv('fullsample.csv')
fullsample_completed = fullsample.loc[fullsample["STATE"]=="COMPLETED"]

In [13]:
fullsample_completed.head(5)

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0
5,31364111_3,COMPLETED,2021-08-17T07:45:07,2021-09-06T16:17:34,16384Mn,9708.04M,24-09:00:00,20-08:32:27,1,1,production,0:0


In [100]:
fullsample_completed['BEGIN'].min()  #Earliest date is Oct. 1,2020 @ 12:03:08 AM
fullsample_completed['END'].max()    #Latest date is Oct. 7, 2021 @ 8:41:11 PM

Timestamp('2021-10-07 20:41:11')

In [101]:
#changing to datetime
fullsample_completed['BEGIN'] = pd.to_datetime(fullsample_completed['BEGIN'])
fullsample_completed['END'] = pd.to_datetime(fullsample_completed['END'])





<ipython-input-101-30ea417736ff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fullsample_completed['BEGIN'] = pd.to_datetime(fullsample_completed['BEGIN'])
<ipython-input-101-30ea417736ff>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fullsample_completed['END'] = pd.to_datetime(fullsample_completed['END'])


In [97]:
fullsample_completed.shape

(7375084, 12)

In [96]:
# hours elapsed from earliest date to latest date is 8924.63 rounded to nearest hundredth 
t1 = fullsample_completed['BEGIN'].min()
t2 = fullsample_completed['END'].max()
duration_in_hr = ((t2 - t1).total_seconds())/3600
round(duration_in_hr,2)

8924.63

In [98]:
# 7,375,084 completed jobs including all jobs from some array
8924.63/7375084

0.00121010553913691